# Métodos Computacionles para las Políticas Públicas
# Proyecto Final - Análisis Envolvente de Datos
## Santiago Ortiz Ortiz

Importamos las librerias necesarias para desarrollar Web Scraping a la página.

In [1]:
import pandas as pd
import re
import requests
import time
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains

**Para ingresar al panel principal.**

La página web a la que le vamos a hacer WS es Sistema de Gestión Hospitalaria (SIHO).
Url principal: https://prestadores.minsalud.gov.co/siho/
Está es la URL principal. Sin embargo, cuando uno quiere examinar la página se da cuenta que tiene unas macro que no se muestran de forma tan simple. Por lo tanto, ingresamos a un link secundario que habilita la macro: https://prestadores.minsalud.gov.co/siho/work.aspx

Una vez acá, el software nos habilita un usuario de invitado. Este ya está digitado, solo hay que darle click en ingresar.

In [2]:
# Creamos una variable con la URL de la página.
url = "https://prestadores.minsalud.gov.co/siho/work.aspx"

# Con ayuda del WebDriver de Chrome vamos a abrir la página.
browser = webdriver.Chrome(executable_path=r"C:\Users\tatoo\Desktop\PhantomJS\Chrome\chromedriver.exe")
# A la hora de hacer Web Scraping es importante dejar un periodo de tiempo relativamente largo (algo que aparente ser humano)
time.sleep(5)
# Con el .get(url), el programa sabe que tiene que ingresar a esa URL
browser.get(url)
time.sleep(5)
# Le pedimos al programa que encuentre un elemento en el código HTML que se llame "btnIngresar" y que le dé CLICK
boton_submit = browser.find_element_by_name("btnIngresar").click()
time.sleep(4)

**Para que Python sepa que esta en otra pestaña**
https://stackoverflow.com/questions/17676036/python-webdriver-to-handle-pop-up-browser-windows-which-is-not-an-alert

In [3]:
# Fijamos una página principal llamada MAIN
main_window_handle = None
while not main_window_handle:
    main_window_handle = browser.current_window_handle
browser.find_element_by_link_text('Capacidad Instalada').click()
signin_window_handle = None
while not signin_window_handle:
    for handle in browser.window_handles:
        if handle != main_window_handle:
            signin_window_handle = handle
            break
browser.switch_to.window(signin_window_handle)

In [ ]:
browser.page_source # Para comprobar que tenga la nueva forma

Lo que hacen los dos códigos siguientes es: 
- Encontrar las casillas año y departamento. Como ambas son elementos de multiple opción este desplegará unos valores.
- Año: 2002 - 2019 (Sin embargo, no hay datos para el 2019)
- Depto: Amazonas - Vaupés (+Bogotá DC.)

In [4]:
# El espacio del año tiene un nombre como el siguiente:
cod_año = browser.find_element_by_name("_ctl0:ContentPlaceHolder1:ddano")
for option in cod_año.find_elements_by_tag_name('option'):
    if option.text == '2003':
        option.click() 
        break

In [5]:
# El espacio del departamento tiene un nombre como el siguiente:
cod_depto = browser.find_element_by_name("_ctl0:ContentPlaceHolder1:dddepa_codigo")
for option in cod_depto.find_elements_by_tag_name('option'):
    if option.text == 'Antioquia':
        option.click() # select() in earlier versions of webdriver
        break

In [6]:
# Después de seleccionar los departamentoS Y LOS AÑOS LE DAMOS CLICK a Buscar
boton_buscar = browser.find_element_by_name("_ctl0:ibBuscarFtr")
boton_buscar.click()

In [7]:
html = browser.execute_script("return document.getElementsByTagName('html')[0].innerHTML")   
soup = BeautifulSoup(html, "html.parser")

In [8]:
soup

<head><title>
	Capacidad Instalada
</title><link href="../includes/appestilos.css" rel="stylesheet"/></head>
<body bgcolor="#ffffff">
<form action="./capacidadinstaladanacional.aspx?pageTitle=Capacidad+Instalada&amp;pageHlp=%2fSIHO%2fayudas%2finformes%2fcapacidad.pdf" id="aspnetForm" method="post" name="aspnetForm">
<input id="__VIEWSTATE" name="__VIEWSTATE" type="hidden" value="/wEPDwULLTIwNDI2NTY5MjEPZBYCZg9kFgICAw9kFggCDw8PFgIeC05hdmlnYXRlVXJsBSMvU0lITy9heXVkYXMvaW5mb3JtZXMvY2FwYWNpZGFkLnBkZmRkAhEPDxYCHgRUZXh0BRNDYXBhY2lkYWQgSW5zdGFsYWRhZGQCEw9kFgYCAQ9kFgJmD2QWAmYPZBYEAgEPEGQQFRIEMjAwMgQyMDAzBDIwMDQEMjAwNQQyMDA2BDIwMDcEMjAwOAQyMDA5BDIwMTAEMjAxMQQyMDEyBDIwMTMEMjAxNAQyMDE1BDIwMTYEMjAxNwQyMDE4BDIwMTkVEgQyMDAyBDIwMDMEMjAwNAQyMDA1BDIwMDYEMjAwNwQyMDA4BDIwMDkEMjAxMAQyMDExBDIwMTIEMjAxMwQyMDE0BDIwMTUEMjAxNgQyMDE3BDIwMTgEMjAxORQrAxJnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dkZAIDDxAPFgYeDURhdGFUZXh0RmllbGQFBm5vbWJyZR4ORGF0YVZhbHVlRmllbGQFBmNvZGlnbx4LXyFEYXRhQm91bmRnZBAVJgAIQW1hem9uYXMJQW50aW9xdWlhBkFyYXVjYTpBcm

In [ ]:
re.findall('<td align="center">(\d*)</td><td>(.*)</td><td align="right">(\d*)</td><td align="right">[\d,]*</td><td align="right">[\d,]*</td>', str(soup))

In [ ]:
agregar = re.findall('<td align="center">(\d*)</td><td>(.*)</td><td align="right">(\d*)</td><td align="right">[\d,]*</td><td align="right">[\d,]*</td>', str(soup))

**Ideas para la base de datos:**
- https://thispointer.com/python-pandas-how-to-add-rows-in-a-dataframe-using-dataframe-append-loc-iloc/

In [ ]:
df_base = pd.DataFrame(columns=['Nivel', 'Concepto', 'Cantidad', 'Departamento', 'Año', 'Clase'])
df_base

In [ ]:
agregar_base = pd.DataFrame(agregar, columns = ['Nivel', 'Concepto', 'Cantidad']) 
agregar_base

In [ ]:
len(agregar_base["Nivel"]) # Esta siempre aparece, para ver cuantos datos son 

In [ ]:
depto = []
for i in range(len(agregar_base["Nivel"])):
    depto.append("Chocó")
depto

In [ ]:
agregar_base["Departamento"] = depto
agregar_base

In [ ]:
anno = []
for i in range(len(agregar_base["Nivel"])):
    anno.append("2003")
anno

In [ ]:
agregar_base["Año"] = anno
agregar_base

In [ ]:
clase = []
for i in range(len(agregar_base["Nivel"])):
    clase.append("Capacidad instalada")
clase

In [ ]:
agregar_base["Clase"] = clase
agregar_base

In [ ]:
df_base = df_base.append(agregar_base, ignore_index=True, sort=False)

In [ ]:
df_base

In [ ]:
base = base.append(prueba, columns = ['Nivel', 'Concepto', 'Cantidad']) 

Hasta acá funciona melo!, pero hay que pensar en una forma de hacer que Python sepa que va a trabajar en la otra ventana
- Revisar: https://stackoverflow.com/questions/32858434/switch-to-popup-in-python-using-selenium

In [ ]:
vacia = []
if len(vacia) == 0:
    vacia = [("NA","NA","NA")]
vacia